# BYU Locating Flagellar Motors

## YOLO Model Training Notebook

This is the third notebook in a series for the BYU Locating Bacterial Flagellar Motors 2025 Kaggle challenge. This notebook handles the training of YOLOv8 object detection models on our prepared dataset.

### Notebook Series:
1. **[Parse Data](https://www.kaggle.com/code/andrewjdarley/parse-data)**: Extracting and preparing 2D slices containing motors to make a YOLO dataset
2. **[Visualize Data](https://www.kaggle.com/code/andrewjdarley/visualize-data)**: Exploratory data analysis and visualization of annotated motor locations
3. **Train YOLO (Current)**: Fine tuning an YOLOv8 object detection model on the prepared dataset
4. **[Submission Notebook](https://www.kaggle.com/code/andrewjdarley/submission-notebook)**: Running inference and generating submission files 

## About this Notebook

This training notebook implements a full YOLOv8 training pipeline for detecting bacterial flagellar motors in tomographic slices. The notebook:

1. **Dataset Configuration**: Sets up and validates the YOLO-format dataset YAML configuration
2. **Model Initialization**: Loads pre-trained YOLOv8 weights for transfer learning
3. **Training Process**: Fine tunes the model with early stopping and periodic checkpoints
4. **Loss Visualization**: Plots training and validation dfl loss curves to monitor progress
5. **Performance Evaluation**: Tests the trained model on random validation samples
6. **Model Export**: Saves the trained weights for use in the submission notebook

In [ ]:
# | Layer | Description        | Local Stride | Cumulative Stride | Output Size |
# | ----- | ------------------ | ------------ | ----------------- | ----------- |
# | Input | Original image     | -            | 1×                | `640 × 640` |
# | 0     | `Conv(64, 3, 2)`   | 2            | 2×                | `320 × 320` |
# | 1     | `Conv(128, 3, 2)`  | 2            | 4× (2×2)          | `160 × 160` |
# | 2     | `C3`               | 1            | 4×                | `160 × 160` |
# | 3     | `Conv(256, 3, 2)`  | 2            | 8× (4×2)          | `80 × 80`   |
# | 5     | `Conv(512, 3, 2)`  | 2            | 16× (8×2)         | `40 × 40`   |
# | 7     | `Conv(1024, 3, 2)` | 2            | 32× (16×2)        | `20 × 20`   |


# | Layer | Operation       | Stride | Output Size           | Note         |
# | ----- | --------------- | ------ | --------------------- | ------------ |
# | 0     | Conv(64, s=2)   | 2×     | 320 × 320             | P1/2         |
# | 1     | Conv(128, s=2)  | 4×     | 160 × 160             | P2/4         |
# | 2     | C3              | 4×     | 160 × 160 (no stride) |              |
# | 3     | Conv(256, s=2)  | 8×     | 80 × 80               | **P3/8** ✅   |
# | 4     | C3              | 8×     | 80 × 80               |              |
# | 5     | Conv(512, s=2)  | 16×    | 40 × 40               | **P4/16** ✅  |
# | 6     | C3              | 16×    | 40 × 40               |              |
# | 7     | Conv(1024, s=2) | 32×    | 20 × 20               | **P5/32** ✅  |
# | 8–10  | etc             | 32×    | 20 × 20               | refined head |

# | Layer | Output Stride | Role  | Connect to YOLO Head   |
# | ----- | ------------- | ----- | ---------------------- |
# | 3     | 8×            | P3/8  | ✅ Yes (small objects)  |
# | 5     | 16×           | P4/16 | ✅ Yes (medium objects) |
# | 7     | 32×           | P5/32 | ✅ Yes (large objects)  |


In [ ]:
# | Backbone            | Params (M) | FLOPs (GFLOPs) | Notes                                                         |
# | ------------------- | ---------- | -------------- | ------------------------------------------------------------- |
# | **CSPDarknet**      | 7–70M      | Low–High       | Native YOLO backbone (used in YOLOv4, YOLOv5)                 |
# | **ResNet-18**       | \~11.7M    | \~1.8          | Shallow, efficient; good for small datasets or edge inference |
# | **ResNet-34**       | \~21.8M    | \~3.6          | Balanced choice if ResNet-18 underperforms                    |
# | **ResNet-50**       | \~25.6M    | \~4.1          | Common backbone for higher accuracy (heavier)                 |
# | **MobileNetV2**     | \~3.4M     | \~0.3          | Extremely lightweight; great for real-time / mobile devices   |
# | **EfficientNet-B0** | \~5.3M     | \~0.39         | High accuracy-per-FLOP; good on small/medium datasets         |
# | **GhostNet**        | \~5M       | Very low       | Ultra-light, highly efficient; used in YOLOv7-Tiny            |
# | **ShuffleNetV2**    | \~2.3M     | Very low       | Designed for low-latency mobile inference                     |
# | **DenseNet-121**    | \~8M       | \~2.9          | Dense connections; better gradient flow but slower            |
# | **ConvNeXt-T**      | \~28M      | \~4.5          | Modern ConvNet with transformer-like performance              |


In [ ]:
#i      (f,     n,      m,  args)
#       from  number module args   args[0] 512   args[1:] rest
#  i      f             t
# m_.i, m_.f,        m_.type 
# c1 c2

0.125 0.0625 0.05


In [ ]:
#   - [-1, 1, Timm, [512, 'efficientnet_b5', True, True, 0, True]]  # Layer 0: returns list of block outputs
#   - [0, 1, Index, [64, 2]]     # Layer 1: P3 = Block 2 output (32x32)
#   - [0, 1, Index, [176, 3]]    # Layer 2: P4 = Block 4 output (16x16)
#   - [0, 1, Index, [512, 4]]    # Layer 3: P5 = Block 6 output (8x8)
#   - [-1, 1, SPPF, [512, 5]]    # Layer 4: SPPF on P5 (8x8) to enhance receptive field

In [9]:
# !python -m build --wheel --no-isolation
# !pip install --upgrade build wheel setuptools

In [2]:
! pip install timm

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 44.7 MB/s eta 0:00:00


In [ ]:
# 方法 A：直接指定文件路径
#!pip install /kaggle/input/ultralytics-thop/ultralytics_thop-2.0.14-py3-none-any.whl --force-reinstall --no-index --no-deps
!pip install /kaggle/input/ultralytics-timm/ultralytics-8.3.133-py3-none-any.whl --no-deps

In [ ]:
!pip install /kaggle/input/yolo-plus/ultralytics-8.3.133-py3-none-any.whl --no-deps

In [ ]:
import torch
import timm

backbone = timm.create_model("resnet50", pretrained=True, features_only=True)
feats = backbone(x)
print([f.shape for f in feats])

# [
#   [1, 64, 112, 112],
#   [1, 256, 56, 56],   # stage 1
#   [1, 512, 28, 28],   # stage 2 -> P3
#   [1, 1024, 14, 14],  # stage 3 -> P4
#   [1, 2048, 7, 7]     # stage 4 -> P5
# ]

In [ ]:
import torch
import timm

model = timm.create_model('resnet50', pretrained=False)
model.eval()
x = torch.randn(1, 3, 256, 256)

def save_hook(name):
    def fn(module, input, output):
        print(f"{name}: {output.shape}")
    return fn

handles = []
for i in range(1, 5):  # layer1~layer4
    layer = getattr(model, f'layer{i}')
    handles.append(layer.register_forward_hook(save_hook(f'layer{i}')))

with torch.no_grad():
    _ = model(x)

for h in handles:
    h.remove()

# layer1: torch.Size([1, 256, 64, 64]) # stage 2 (C2)
# layer2: torch.Size([1, 512, 32, 32]) # stage 3 (C3)
# layer3: torch.Size([1, 1024, 16, 16]) # stage 4 (C4)
# layer4: torch.Size([1, 2048, 8, 8]) # stage 5 (C5)

# | YOLO Head Input | ResNet Stage | Output Shape        | Notes                   |
# | --------------- | ------------ | ------------------- | ----------------------- |
# | P3/8            | layer2       | `[1, 512, 32, 32]`  | Spatial downscale = 8×  | => 256 / 32 >= 8x
# | P4/16           | layer3       | `[1, 1024, 16, 16]` | Spatial downscale = 16× | => 256 / 32 >= 16x
# | P5/32           | layer4       | `[1, 2048, 8, 8]`   | Spatial downscale = 32× | 


layer1: torch.Size([1, 256, 64, 64])
layer2: torch.Size([1, 512, 32, 32])
layer3: torch.Size([1, 1024, 16, 16])
layer4: torch.Size([1, 2048, 8, 8])


In [4]:
import torch
import timm

model = timm.create_model('convnext_tiny', features_only=True, pretrained=True)
model.eval()
x = torch.randn(1, 3, 256, 256)

with torch.no_grad():
    feats = model(x)
    for i, f in enumerate(feats):
        print(f"stage{i}: {f.shape}")


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

stage0: torch.Size([1, 96, 64, 64])
stage1: torch.Size([1, 192, 32, 32])
stage2: torch.Size([1, 384, 16, 16])
stage3: torch.Size([1, 768, 8, 8])


c:\Users\Freedomkwok2022\ML_Learn\20241127_Course_01_JS2024_Baseline\env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Freedomkwok2022\.cache\huggingface\hub\models--timm--convnext_tiny.in12k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
import torch
import timm

import torch
import timm

model = timm.create_model('swinv2_tiny_window8_256', pretrained=False)
model.eval()
x = torch.randn(1, 3, 256, 256)

def save_hook(name):
    def fn(module, input, output):
        print(f"{name}: {output.shape}")
    return fn

handles = []
# Swin V2 的主干是 model.layers[0], model.layers[1], model.layers[2], model.layers[3]
for i in range(4):
    handles.append(model.layers[i].register_forward_hook(save_hook(f'layers[{i}]')))

with torch.no_grad():
    _ = model(x)

for h in handles:
    h.remove()



In [14]:
import torch
import timm

model = timm.create_model('efficientnet_b5', pretrained=False)
model.eval()
x = torch.randn(1, 3, 256, 256)

# 用 hook 方式输出每个 block 的 shape
def save_hook(name):
    def fn(module, input, output):
        print(f"{name}: {output.shape}")
    return fn

handles = []
for i, block in enumerate(model.blocks):
    handles.append(block.register_forward_hook(save_hook(f"Block {i}")))

# 正确做法：直接把 x 输入整个 model
with torch.no_grad():
    _ = model(x)

for h in handles:
    h.remove()

# Block 0: torch.Size([1, 24, 128, 128])
# Block 1: torch.Size([1, 40, 64, 64]) 
# Block 2: torch.Size([1, 64, 32, 32]) 8x
# Block 3: torch.Size([1, 128, 16, 16])
# Block 4: torch.Size([1, 176, 16, 16]) 16
# Block 5: torch.Size([1, 304, 8, 8])
# Block 6: torch.Size([1, 512, 8, 8])  32

Block 0: torch.Size([1, 24, 128, 128])
Block 1: torch.Size([1, 40, 64, 64])
Block 2: torch.Size([1, 64, 32, 32])
Block 3: torch.Size([1, 128, 16, 16])
Block 4: torch.Size([1, 176, 16, 16])
Block 5: torch.Size([1, 304, 8, 8])
Block 6: torch.Size([1, 512, 8, 8])


In [ ]:
import torch
import timm

model = timm.create_model('efficientnet_b2', pretrained=False)
model.eval()
x = torch.randn(1, 3, 256, 256)

# 用 hook 方式输出每个 block 的 shape
def save_hook(name):
    def fn(module, input, output):
        print(f"{name}: {output.shape}")
    return fn

handles = []
for i, block in enumerate(model.blocks):
    handles.append(block.register_forward_hook(save_hook(f"Block {i}")))

# 正确做法：直接把 x 输入整个 model
with torch.no_grad():
    _ = model(x)

for h in handles:
    h.remove()

In [ ]:
# | Stage | Layer | Operation  | Input Layer | EfficientNet Block | Channels (EffNet) | Spatial Size | YOLOv11n Layer(s) | Channels (YOLOv11n) | Notes                    |
# | ----- | ----- | ---------- | ----------- | ------------------ | ----------------- | ------------ | ----------------- | ------------------- | ------------------------ |
# | P3    | 1     | `Index[2]` | Layer 0     | Block 2            | 64                | 32×32        | Layer 3           | 256                 | First detect scale       |
# | P4    | 2     | `Index[4]` | Layer 0     | Block 4            | 176               | 16×16        | Layers 4–6        | 512                 | Mid detect scale         |
# | P5    | 3     | `Index[6]` | Layer 0     | Block 6            | 512               | 8×8          | Layers 7–9        | 1024                | Deepest feature          |
# | P5+   | 4     | `SPPF`     | Layer 3     | —                  | 512               | 8×8          | Layer 9           | 1024                | Expanded receptive field |

# | Feature          | EffNet-B5 Head                                           | YOLOv11n Head                                   | Notes                                          |
# | ---------------- | -------------------------------------------------------- | ----------------------------------------------- | ---------------------------------------------- |
# | **P5** (deepest) | `SPPF` output (8×8) = Layer 4                            | Layer 10: Backbone P5 (20×20 → 8×8)             | Both start from 8×8                            |
# | → refine P5      | `C2f[512]` after concat                                  | `C3k2[1024]` after concat (Layer 22)            | YOLOv11n uses 1024, EffNet uses 512            |
# | **P4** (middle)  | `Upsample`, concat with P4 (Layer 2 = 16×16), `C2f[176]` | `Upsample`, concat Layer 6, `C3k2[512]`         | EffNet uses 176, YOLOv11n uses 512             |
# | → refine again   | Downsample + concat (32→16) + `C2f[176]` again           | Downsample + concat → `C3k2[512]` again         | Both re-process P4 mid-level fusion            |
# | **P3** (shallow) | `Upsample`, concat with P3 (Layer 1 = 32×32), `C2f[64]`  | `Upsample`, concat Layer 4, `C3k2[256]`         | EffNet uses 64, YOLOv11n uses 256              |
# | → refine again   | Downsample + concat (→16×16) + `C2f[176]`                | Downsample + concat → `C3k2[512]`               | Second P3 fusion → P4 again                    |
# | **Detection**    | `Detect[64, 176, 512]` (layers 10,13,16)                 | `YOLOEDetect[256, 512, 1024]` (layers 16,19,22) | Channels are projected smaller in EffNet setup |


# efficientnet_b5
# Block 0: torch.Size([1, 24, 128, 128])
# Block 1: torch.Size([1, 40, 64, 64])
# Block 2: torch.Size([1, 64, 32, 32])   # Index 2 x 8
# Block 3: torch.Size([1, 128, 16, 16])  
# Block 4: torch.Size([1, 176, 16, 16])   # Index 4 x16
# Block 5: torch.Size([1, 304, 8, 8])
# Block 6: torch.Size([1, 512, 8, 8])    # Index 6 x 32

# 2. 然后正常 import 并构建  
from ultralytics import YOLO
# 1. 你的 YAML 配置字符串
# import timm.layers.patch_embed as _pe
# _pe._assert = lambda cond, msg=None: None
eff_yolo_config = """
nc: 1  # number of classes

backbone:
  - [-1, 1, Timm, [512, 'efficientnet_b5', True, True, 0, True]]  # Layer 0: returns list of block outputs
  - [0, 1, Index, [64, 2]]     # Layer 1: P3 = Block 2 output (32x32)
  - [0, 1, Index, [176, 4]]    # Layer 2: P4 = Block 4 output (16x16)
  - [0, 1, Index, [512, 6]]    # Layer 3: P5 = Block 6 output (8x8)
  - [-1, 1, SPPF, [512, 5]]    # Layer 4: SPPF on P5 (8x8) to enhance receptive field

head:
  # 上采样/拼接/检测头，通道数建议与 backbone 输出保持一致
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]            # 5, SPPF上采样 (8->16)
  - [[-1, 2], 1, Concat, [1]]                             # 6, 拼接16x16的两个特征
  - [-1, 3, C2f, [176]]                                   # 7, 通道数和P4对齐

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]            # 8, 上采样 (16->32)
  - [[-1, 1], 1, Concat, [1]]                             # 9, 拼接32x32的两个特征
  - [-1, 3, C2f, [64]]                                    # 10, 通道数和P3对齐

  - [-1, 1, Conv, [64, 3, 2]]                             # 11, 下采样 (32->16)
  - [[-1, 7], 1, Concat, [1]]                             # 12, 拼接16x16的两个特征
  - [-1, 3, C2f, [176]]                                   # 13

  - [-1, 1, Conv, [176, 3, 2]]                            # 14, 下采样 (16->8)
  - [[-1, 4], 1, Concat, [1]]                             # 15, 拼接8x8的两个特征
  - [-1, 3, C2f, [512]]                                   # 16

  - [[10, 13, 16], 1, Detect, [nc]]                       # 17, 检测头, 多尺度
"""

swin_yolo_config = """
nc: 80 # number of classes
backbone:
  - [-1, 1, Timm, [352, 'efficientnet_b2', True, True, 0, True]]
  - [0, 1, Index, [48, 2]]    # features[2]  [1, 64, 32, 32]
  - [0, 1, Index, [120, 3]]   # features[4]  [1, 176, 16, 16]
  - [0, 1, Index, [352, 4]]   # features[6]  [1, 512, 8, 8]
  - [-1, 1, SPPF, [352, 5]]

head:
  # 上采样/拼接/检测头，通道数建议与 backbone 输出保持一致
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]            # 5, SPPF上采样 (8->16)
  - [[-1, 2], 1, Concat, [1]]                             # 6, 拼接16x16的两个特征
  - [-1, 3, C2f, [120]]                                   # 7, 通道数和P4对齐

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]            # 8, 上采样 (16->32)
  - [[-1, 1], 1, Concat, [1]]                             # 9, 拼接32x32的两个特征
  - [-1, 3, C2f, [48]]                                    # 10, 通道数和P3对齐

  - [-1, 1, Conv, [64, 3, 2]]                             # 11, 下采样 (32->16)
  - [[-1, 7], 1, Concat, [1]]                             # 12, 拼接16x16的两个特征
  - [-1, 3, C2f, [120]]                                   # 13

  - [-1, 1, Conv, [176, 3, 2]]                            # 14, 下采样 (16->8)
  - [[-1, 4], 1, Concat, [1]]                             # 15, 拼接8x8的两个特征
  - [-1, 3, C2f, [352]]                                   # 16

  - [[10, 13, 16], 1, Detect, [nc]]                       # 17, 检测头, 多尺度

"""


# 2. 写入到本地文件
yaml_path= "swin_yolo.yaml"
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(swin_yolo_config)

# 3. 直接用文件路径加载模型结构
model = YOLO(yaml_path,verbose=False)





In [ ]:

from ultralytics import YOLO
swin_yolo_config = """
# Parameters
nc: 80  # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]  # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024]  # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768]   # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512]   # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512]   # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 1]]  # 1
  - [-1, 1, space_to_depth,[1]] # 2-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 1]]  # 4
  - [-1, 1, space_to_depth,[1]] # 5-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 1]]  # 7
  - [-1, 1, space_to_depth,[1]] # 8-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 1]]  # 10
  - [-1, 1, space_to_depth,[1]] # 11-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 13

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 9], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 16

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 19 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 16], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 22 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 13], 1, Concat, [1]]  # cat head P5
  - [-1, 3, C2f, [1024]]  # 25 (P5/32-large)

  - [[19, 22, 25], 1, Detect, [nc]]  # Detect(P3, P4, P5)


"""

yaml_path= "swin_yolo.yaml"
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(swin_yolo_config)

# 3. 直接用文件路径加载模型结构
model = YOLO(yaml_path,verbose=False)

In [ ]:
from ultralytics import YOLO
swin_yolo_config = """
nc: 10  # number of classes
depth_multiple: 0.33  # scales module repeats
width_multiple: 1.00  # scales convolution channels

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [16, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [32, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [32, True]]
  - [-1, 1, Conv, [64, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [64, True]]
  - [-1, 1, Conv, [128, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [256, True]]
  - [-1, 1, SPPF, [256, 5]]  # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [128]]  # 12

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [64]]  # 15 (P3/8-small)

  - [[15], 1, DyDetect, [nc]]  # Detect(P3, P4, P5)

"""

# yaml_path= "swin_yolo.yaml"
# with open(yaml_path, "w", encoding="utf-8") as f:
#     f.write(swin_yolo_config)

# # 3. 直接用文件路径加载模型结构
# model = YOLO(yaml_path,verbose=False)

In [ ]:
def ensure_channels_first(x):
    # x: [B, H, W, C] or [B, C, H, W]
    if x.dim() == 4 and x.shape[1] < 10 and x.shape[-1] > 10:
        return x.permute(0, 3, 1, 2).contiguous()
    return x


In [ ]:
model = YOLO(yaml_path,verbose=False)

In [ ]:

import os
import torch
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
# from ultralytics import YOLO
import yaml
import pandas as pd
import json

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

# Define paths for Kaggle environment
yolo_dataset_dir = "/kaggle/input/parse-data/yolo_dataset"
yolo_weights_dir = "/kaggle/working/yolo_weights"
yolo_pretrained_weights = "/kaggle/input/yolo11/pytorch/default/1/yolo11n.pt"  # Path to pre-downloaded weights

# Create weights directory if it doesn't exist
os.makedirs(yolo_weights_dir, exist_ok=True)

def fix_yaml_paths(yaml_path):
    """
    Fix the paths in the YAML file to match the actual Kaggle directories
    
    Args:
        yaml_path (str): Path to the original dataset YAML file
        
    Returns:
        str: Path to the fixed YAML file
    """
    print(f"Fixing YAML paths in {yaml_path}")
    
    # Read the original YAML
    with open(yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    
    # Update paths to use actual dataset location
    if 'path' in yaml_data:
        yaml_data['path'] = yolo_dataset_dir
    
    # Create a new fixed YAML in the working directory
    fixed_yaml_path = "/kaggle/working/fixed_dataset.yaml"
    with open(fixed_yaml_path, 'w') as f:
        yaml.dump(yaml_data, f)
    
    print(f"Created fixed YAML at {fixed_yaml_path} with path: {yaml_data.get('path')}")
    return fixed_yaml_path

def plot_dfl_loss_curve(run_dir):
    """
    Plot the DFL loss curves for train and validation, marking the best model
    
    Args:
        run_dir (str): Directory where the training results are stored
    """
    # Path to the results CSV file
    results_csv = os.path.join(run_dir, 'results.csv')
    
    if not os.path.exists(results_csv):
        print(f"Results file not found at {results_csv}")
        return
    
    # Read results CSV
    results_df = pd.read_csv(results_csv)
    
    # Check if DFL loss columns exist
    train_dfl_col = [col for col in results_df.columns if 'train/dfl_loss' in col]
    val_dfl_col = [col for col in results_df.columns if 'val/dfl_loss' in col]
    
    if not train_dfl_col or not val_dfl_col:
        print("DFL loss columns not found in results CSV")
        print(f"Available columns: {results_df.columns.tolist()}")
        return
    
    train_dfl_col = train_dfl_col[0]
    val_dfl_col = val_dfl_col[0]
    
    # Find the epoch with the best validation loss
    best_epoch = results_df[val_dfl_col].idxmin()
    best_val_loss = results_df.loc[best_epoch, val_dfl_col]
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    # Plot training and validation losses
    plt.plot(results_df['epoch'], results_df[train_dfl_col], label='Train DFL Loss')
    plt.plot(results_df['epoch'], results_df[val_dfl_col], label='Validation DFL Loss')
    
    # Mark the best model with a vertical line
    plt.axvline(x=results_df.loc[best_epoch, 'epoch'], color='r', linestyle='--', 
                label=f'Best Model (Epoch {int(results_df.loc[best_epoch, "epoch"])}, Val Loss: {best_val_loss:.4f})')
    
    # Add labels and legend
    plt.xlabel('Epoch')
    plt.ylabel('DFL Loss')
    plt.title('Training and Validation DFL Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Save the plot in the same directory as weights
    plot_path = os.path.join(run_dir, 'dfl_loss_curve.png')
    plt.savefig(plot_path)
    
    # Also save it to the working directory for easier access
    plt.savefig(os.path.join('/kaggle/working', 'dfl_loss_curve.png'))
    
    print(f"Loss curve saved to {plot_path}")
    plt.close()
    
    # Return the best epoch info
    return best_epoch, best_val_loss

def train_yolo_model(yaml_path, pretrained_weights_path, epochs=30, batch_size=4, img_size=640):
    """
    Train a YOLO model on the prepared dataset
    
    Args:
        yaml_path (str): Path to the dataset YAML file
        pretrained_weights_path (str): Path to pre-downloaded weights file
        epochs (int): Number of training epochs
        batch_size (int): Batch size for training
        img_size (int): Image size for training
    """
    print(f"Loading pre-trained weights from: {pretrained_weights_path}")
    
    # Load a pre-trained YOLOv8 model
    #model = YOLO(pretrained_weights_path)
    model = YOLO("swin_yolo.yaml")
    # Train the model with early stopping
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=yolo_weights_dir,
        name='motor_detector',
        exist_ok=True,
        patience=5,              # Early stopping if no improvement for 5 epochs
        save_period=5,           # Save checkpoints every 5 epochs
        val=True,                # Ensure validation is performed
        verbose=True             # Show detailed output during training
    )
    
    # Get the path to the run directory
    run_dir = os.path.join(yolo_weights_dir, 'motor_detector')
    
    # Plot and save the loss curve
    best_epoch_info = plot_dfl_loss_curve(run_dir)
    
    if best_epoch_info:
        best_epoch, best_val_loss = best_epoch_info
        print(f"\nBest model found at epoch {best_epoch} with validation DFL loss: {best_val_loss:.4f}")
    
    return model, results

def predict_on_samples(model, num_samples=4):
    """
    Run predictions on random validation samples and display results
    
    Args:
        model: Trained YOLO model
        num_samples (int): Number of random samples to test
    """
    # Get validation images
    val_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
    if not os.path.exists(val_dir):
        print(f"Validation directory not found at {val_dir}")
        # Try train directory instead if val doesn't exist
        val_dir = os.path.join(yolo_dataset_dir, 'images', 'train')
        print(f"Using train directory for predictions instead: {val_dir}")
        
    if not os.path.exists(val_dir):
        print("No images directory found for predictions")
        return
    
    val_images = os.listdir(val_dir)
    
    if len(val_images) == 0:
        print("No images found for prediction")
        return
    
    # Select random samples
    num_samples = min(num_samples, len(val_images))
    samples = random.sample(val_images, num_samples)
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, img_file in enumerate(samples):
        if i >= len(axes):
            break
            
        img_path = os.path.join(val_dir, img_file)
        
        # Run prediction
        results = model.predict(img_path, conf=0.25)[0]
        
        # Load and display the image
        img = Image.open(img_path)
        axes[i].imshow(np.array(img), cmap='gray')
        
        # Draw ground truth box if available (from filename)
        try:
            # This assumes your filenames contain coordinates in a specific format
            parts = img_file.split('_')
            y_part = [p for p in parts if p.startswith('y')]
            x_part = [p for p in parts if p.startswith('x')]
            
            if y_part and x_part:
                y_gt = int(y_part[0][1:])
                x_gt = int(x_part[0][1:].split('.')[0])
                
                box_size = 24
                rect_gt = Rectangle((x_gt - box_size//2, y_gt - box_size//2), 
                              box_size, box_size, 
                              linewidth=1, edgecolor='g', facecolor='none')
                axes[i].add_patch(rect_gt)
        except:
            pass  # Skip ground truth if parsing fails
        
        # Draw predicted boxes (red)
        if len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            confs = results.boxes.conf.cpu().numpy()
            
            for box, conf in zip(boxes, confs):
                x1, y1, x2, y2 = box
                rect_pred = Rectangle((x1, y1), x2-x1, y2-y1, 
                                     linewidth=1, edgecolor='r', facecolor='none')
                axes[i].add_patch(rect_pred)
                axes[i].text(x1, y1-5, f'{conf:.2f}', color='red')
        
        axes[i].set_title(f"Image: {img_file}\nGround Truth (green) vs Prediction (red)")
    
    plt.tight_layout()
    
    # Save the predictions plot
    plt.savefig(os.path.join('/kaggle/working', 'predictions.png'))
    plt.show()

# Check and create a dataset YAML if needed
def prepare_dataset():
    """
    Check if dataset exists and create a proper YAML if needed
    
    Returns:
        str: Path to the YAML file to use for training
    """
    # Check if images exist
    train_images_dir = os.path.join(yolo_dataset_dir, 'images', 'train')
    val_images_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
    train_labels_dir = os.path.join(yolo_dataset_dir, 'labels', 'train')
    val_labels_dir = os.path.join(yolo_dataset_dir, 'labels', 'val')
    
    # Print directory existence status
    print(f"Directory status:")
    print(f"- Train images dir exists: {os.path.exists(train_images_dir)}")
    print(f"- Val images dir exists: {os.path.exists(val_images_dir)}")
    print(f"- Train labels dir exists: {os.path.exists(train_labels_dir)}")
    print(f"- Val labels dir exists: {os.path.exists(val_labels_dir)}")
    
    # Check for original YAML file
    original_yaml_path = os.path.join(yolo_dataset_dir, 'dataset.yaml')
    
    if os.path.exists(original_yaml_path):
        print(f"Found original dataset.yaml at {original_yaml_path}")
        # Fix the paths in the YAML
        return fix_yaml_paths(original_yaml_path)
    else:
        print(f"Original dataset.yaml not found, creating a new one")
        
        # Create a new YAML file
        yaml_data = {
            'path': yolo_dataset_dir,
            'train': 'images/train',
            'val': 'images/train' if not os.path.exists(val_images_dir) else 'images/val',
            'names': {0: 'motor'}
        }
        
        new_yaml_path = "/kaggle/working/dataset.yaml"
        with open(new_yaml_path, 'w') as f:
            yaml.dump(yaml_data, f)
            
        print(f"Created new YAML at {new_yaml_path}")
        return new_yaml_path

# Main execution
def main():
    print("Starting YOLO training process...")
    
    # Prepare dataset and get YAML path
    yaml_path = prepare_dataset()
    print(f"Using YAML file: {yaml_path}")
    
    # Print YAML file contents
    with open(yaml_path, 'r') as f:
        yaml_content = f.read()
    print(f"YAML file contents:\n{yaml_content}")
    
    # Train model
    print("\nStarting YOLO training...")
    model, results = train_yolo_model(
        yaml_path,
        pretrained_weights_path=yolo_pretrained_weights,
        epochs=30  # Using 30 epochs instead of 100 for faster training
    )
    
    print("\nTraining complete!")
    
    # Run predictions
    print("\nRunning predictions on sample images...")
    predict_on_samples(model, num_samples=4)

if __name__ == "__main__":
    main()